In [19]:
import bz2
import os
import numpy as np
%matplotlib inline

import conllu
import pandas as pd
import numpy as np
import spacy
import en_core_web_lg
import collections
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.model_selection import cross_validate
from sklearn.metrics import classification_report, make_scorer, precision_recall_fscore_support
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from tqdm import tqdm

In [3]:
from keras.models import Sequential
from keras.layers import Dense

In [4]:
model = Sequential()

In [5]:
model.add(Dense(units=64, activation='relu', input_dim=100))
model.add(Dense(units=10, activation='softmax'))

In [6]:
model.compile(loss='categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

In [25]:
# Load embeddings
embedding_index = {}
f = bz2.open("/Users/serhiinechyporhuk/courses/ubercorpus.cased.tokenized.glove.300d.bz2")
for i,line in enumerate(f):
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embedding_index[word] = coefs
f.close()

In [13]:
print("Number of word embeddings:", len(embedding_index))

Number of word embeddings: 595119


In [17]:
train = conllu.parse(open("uk_iu-ud-train.conllu").read())
test = conllu.parse(open("uk_iu-ud-test.conllu").read())
dev = conllu.parse(open("uk_iu-ud-dev.conllu").read())

In [111]:
train[0]

[OrderedDict([('id', 1),
              ('form', 'У'),
              ('lemma', 'у'),
              ('upostag', 'ADP'),
              ('xpostag', 'Spsl'),
              ('feats', OrderedDict([('Case', 'Loc')])),
              ('head', 2),
              ('deprel', 'case'),
              ('deps', None),
              ('misc', OrderedDict([('Id', '0003')]))]),
 OrderedDict([('id', 2),
              ('form', 'домі'),
              ('lemma', 'дім'),
              ('upostag', 'NOUN'),
              ('xpostag', 'Ncmsln'),
              ('feats',
               OrderedDict([('Animacy', 'Inan'),
                            ('Case', 'Loc'),
                            ('Gender', 'Masc'),
                            ('Number', 'Sing')])),
              ('head', 6),
              ('deprel', 'obl'),
              ('deps', None),
              ('misc', OrderedDict([('Id', '0004')]))]),
 OrderedDict([('id', 3),
              ('form', 'римського'),
              ('lemma', 'римський'),
              ('up

In [215]:
word_index = {}
labels_uniq = set()
i = 0
for row in train:
    for odict in row:
        if odict['form'] not in word_index:
            word_index[odict['form']] = i
            i+=1
        labels_uniq.add(odict['deprel'])
print("Length of dictionary:", len(word_index))
print("Unique labels:", len(labels_uniq))

Length of dictionary: 23976
Unique labels: 49


In [216]:
labels = dict(zip(sorted(labels_uniq), range(len(labels_uniq))))

In [226]:
from keras.utils import to_categorical
def prepare(rows):
    X_train1 = []
    X_train2 = []
    y_train = []
    for row in rows:
        for odict in row:
            if odict['head'] > 0 and odict['deprel'] in labels:
                child = odict['form']
                head = row[odict['head']-1]['form']
                child_idx = word_index.get(child, VOCAB_LENGTH)
                head_idx = word_index.get(head, VOCAB_LENGTH)
                X_train1.append(np.asarray([child_idx]),)
                X_train2.append(np.asarray([head_idx]))
                y_train.append(labels[odict['deprel']])
    return (np.asarray(X_train1), np.asarray(X_train2)), to_categorical(y_train)

(X_train1, X_train2), y_train = prepare(train)

In [208]:
VECT_LENGTH=300
VOCAB_LENGTH = len(word_index)
INPUT_LENGTH = 2
OUTPUT_LENGTH = len(labels_uniq)

In [209]:
VOCAB_LENGTH, OUTPUT_LENGTH, len(X_train1)

(23976, 49, 70585)

In [210]:
embedding_matrix = np.zeros((VOCAB_LENGTH+1, VECT_LENGTH))
for word,i in word_index.items():
    embedding_vector = embedding_index.get(word.encode('utf8'))
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [211]:
embedding_matrix.shape

(23977, 300)

In [219]:
from keras import backend as K

def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))


In [220]:
from keras.layers import Input, Embedding, Concatenate, Flatten
from keras.models import Model

w1 = Input(shape=(1,), dtype='int32', name='w1')
embed_w1 = Embedding(
    VOCAB_LENGTH+1,
    VECT_LENGTH,
    weights=[embedding_matrix],
    input_length=1,
    trainable=False
)(w1)

w2 = Input(shape=(1,), dtype='int32', name='w2')
embed_w2 = Embedding(
    VOCAB_LENGTH+1,
    VECT_LENGTH,
    weights=[embedding_matrix],
    input_length=1,
    trainable=False
)(w2)


x = Concatenate(axis=-1, name='concat')([embed_w1, embed_w2])
x = Dense(1000, activation='relu', name='dense1')(x)
x = Flatten()(x)
x = Dense(OUTPUT_LENGTH, activation='softmax', name='outputl')(x)

model = Model(inputs=[w1, w2], outputs=x)
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc', f1])
#model = Model(w2,embed_w2)

#model = Sequential()
#model.add(embed_w1)

In [229]:
model.fit(x={'w1': X_train1, 'w2': X_train2}, y=y_train, epochs=5,)

Epoch 1/5
70585/70585 [==============================] - 22s 315us/step - loss: 0.7913 - acc: 0.7912 - f1: 0.7894
Epoch 2/5
70585/70585 [==============================] - 22s 315us/step - loss: 0.5881 - acc: 0.8442 - f1: 0.8444
Epoch 3/5
70585/70585 [==============================] - 22s 318us/step - loss: 0.4476 - acc: 0.8846 - f1: 0.8848
Epoch 4/5
70585/70585 [==============================] - 23s 326us/step - loss: 0.3619 - acc: 0.9098 - f1: 0.9102
Epoch 5/5
70585/70585 [==============================] - 23s 326us/step - loss: 0.3176 - acc: 0.9252 - f1: 0.9253


In [227]:
(X_test1, X_test2), y_test = prepare(test)

In [230]:
model.evaluate(x={'w1': X_test1, 'w2': X_test2}, y=y_test)

14154/14154 [==============================] - 1s 96us/step


[2.1283637827797364, 0.6005369506768964, 0.6007689974040858]